# 测试集图像分类预测结果

使用训练好的图像分类模型，预测测试集的所有图像，得到预测结果表格。

同济子豪兄：https://space.bilibili.com/1900783

[代码运行云GPU环境](https://featurize.cn/?s=d7ce99f842414bfcaea5662a97581bd1)：GPU RTX 3060、CUDA v11.2

## 导入工具包

In [2]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn.functional as F

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 图像预处理

In [3]:
from torchvision import transforms

# # 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
# train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
#                                       transforms.RandomHorizontalFlip(),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#                                      ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## 载入测试集（和训练代码教程相同）

In [4]:
# 数据集文件夹路径
dataset_dir = 'D:\\dataset\\c100sp'
test_path = os.path.join(dataset_dir, 'val')
from torchvision import datasets
# 载入测试集
test_dataset = datasets.ImageFolder(test_path, test_transform)
print('测试集图像数量', len(test_dataset))
print('类别个数', len(test_dataset.classes))
print('各类别名称', test_dataset.classes)
# 载入类别名称 和 ID索引号 的映射字典
idx_to_labels = np.load('idx_to_labels.npy', allow_pickle=True).item()
# 获得类别名称
classes = list(idx_to_labels.values())
print(classes)

测试集图像数量 9978
类别个数 100
各类别名称 ['云', '仓鼠', '兔子', '兰花', '割草机', '向日葵', '城堡', '大海', '大象', '女人', '女孩', '小婴儿', '山', '巴士', '平原', '床', '恐龙', '房屋', '拖拉机', '摩天大楼', '摩托车', '时钟', '有轨电车', '杯子', '松树', '松鼠', '枫树', '柳树', '桌子', '桥', '梨', '棕榈树', '森林', '椅子', '橘子', '橡树', '比目鱼', '毛虫', '水族馆鱼类', '水濑', '沙发', '浣熊', '海狸', '海豚', '海豹', '海龟', '火箭', '火车', '灯', '熊', '牛', '狐狸', '狮子', '狼', '玫瑰', '瓶子', '甜椒', '甲虫', '电视', '电话', '男人', '男孩', '皮卡', '盘子', '碗', '箱子', '罂粟', '老虎', '老鼠', '自行车', '臭鼬', '苹果', '蘑菇', '蛇', '蜗牛', '蜘蛛', '蜜蜂', '蜥蜴', '蝴蝶', '螃蟹', '蟑螂', '蠕虫', '衣柜', '袋鼠', '豪猪', '豹', '负鼠', '郁金香', '金属罐', '键盘', '马路', '骆驼', '鲨鱼', '鲸鱼', '鳄鱼', '鳐', '鳟鱼', '黑猩猩', '鼩鼱', '龙虾']
['云', '仓鼠', '兔子', '兰花', '割草机', '向日葵', '城堡', '大海', '大象', '女人', '女孩', '小婴儿', '山', '巴士', '平原', '床', '恐龙', '房屋', '拖拉机', '摩天大楼', '摩托车', '时钟', '有轨电车', '杯子', '松树', '松鼠', '枫树', '柳树', '桌子', '桥', '梨', '棕榈树', '森林', '椅子', '橘子', '橡树', '比目鱼', '毛虫', '水族馆鱼类', '水濑', '沙发', '浣熊', '海狸', '海豚', '海豹', '海龟', '火箭', '火车', '灯', '熊', '牛', '狐狸', '狮子', '狼', '玫瑰', '瓶子', '甜椒', '甲虫', '电视', '电话', 

## 导入训练好的模型

In [5]:
model = torch.load('best-0.867.pth')
model = model.eval().to(device)

## 表格A-测试集图像路径及标注

In [6]:
test_dataset.imgs[:10]

[('D:\\dataset\\c100sp\\val\\云\\1.jpg', 0),
 ('D:\\dataset\\c100sp\\val\\云\\104.jpg', 0),
 ('D:\\dataset\\c100sp\\val\\云\\114.jpg', 0),
 ('D:\\dataset\\c100sp\\val\\云\\117.jpg', 0),
 ('D:\\dataset\\c100sp\\val\\云\\119.jpg', 0),
 ('D:\\dataset\\c100sp\\val\\云\\131.jpeg', 0),
 ('D:\\dataset\\c100sp\\val\\云\\14.jpeg', 0),
 ('D:\\dataset\\c100sp\\val\\云\\142.jpg', 0),
 ('D:\\dataset\\c100sp\\val\\云\\145.jpg', 0),
 ('D:\\dataset\\c100sp\\val\\云\\150.jpg', 0)]

In [7]:
img_paths = [each[0] for each in test_dataset.imgs]

In [8]:
df = pd.DataFrame()
df['图像路径'] = img_paths
df['标注类别ID'] = test_dataset.targets
df['标注类别名称'] = [idx_to_labels[ID] for ID in test_dataset.targets]

In [9]:
df.to_csv('测试集标注.csv', index=False)

## 表格B-测试集每张图像的图像分类预测结果，以及各类别置信度

In [10]:
# 记录 top-n 预测结果
n = 3

In [11]:
df_pred = pd.DataFrame()
for idx, row in tqdm(df.iterrows()):
    img_path = row['图像路径']
    img_pil = Image.open(img_path).convert('RGB')
    input_img = test_transform(img_pil).unsqueeze(0).to(device) # 预处理
    pred_logits = model(input_img) # 执行前向预测，得到所有类别的 logit 预测分数
    pred_softmax = F.softmax(pred_logits, dim=1) # 对 logit 分数做 softmax 运算

    pred_dict = {}

    top_n = torch.topk(pred_softmax, n) # 取置信度最大的 n 个结果
    pred_ids = top_n[1].cpu().detach().numpy().squeeze() # 解析出类别
    
    # top-n 预测结果
    for i in range(1, n+1):
        pred_dict['top-{}-预测ID'.format(i)] = pred_ids[i-1]
        pred_dict['top-{}-预测名称'.format(i)] = idx_to_labels[pred_ids[i-1]]
    pred_dict['top-n预测正确'] = row['标注类别ID'] in pred_ids
    # 每个类别的预测置信度
    for idx, each in enumerate(classes):
        pred_dict['{}-预测置信度'.format(each)] = pred_softmax[0][idx].cpu().detach().numpy()
        
    df_pred = df_pred._append(pred_dict, ignore_index=True)     

305it [00:23, 12.84it/s]C:\Users\POLARIS\.conda\envs\nsuai\lib\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
2404it [03:13, 11.97it/s]C:\Users\POLARIS\.conda\envs\nsuai\lib\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
2440it [03:16, 10.47it/s]C:\Users\POLARIS\.conda\envs\nsuai\lib\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
3427it [04:50, 12.49it/s]C:\Users\POLARIS\.conda\envs\nsuai\lib\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
8789it [14:35,  7.81it/s]C:\Users\POLARIS\.conda\envs\nsuai\lib\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency express

## 拼接AB两张表格

In [12]:
df = pd.concat([df, df_pred], axis=1)

## 导出完整表格

In [13]:
df.to_csv('测试集预测结果.csv',encoding="utf_8_sig", index=False)